In [99]:
import pandas as pd
import numpy as np

In [100]:
kg = pd.read_csv('./Kanagawa_22.csv')

In [101]:
kg.shape

(349894, 29)

In [102]:
kg.shape

(349894, 29)

In [103]:
kg['Transaction-price(total)'].head()

0     8700000
1    38000000
2    22000000
3    31000000
4    26000000
Name: Transaction-price(total), dtype: int64

In [104]:
kg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 349894 entries, 0 to 349893
Data columns (total 29 columns):
 #   Column                              Non-Null Count   Dtype  
---  ------                              --------------   -----  
 0   No                                  349894 non-null  int64  
 1   Type                                349894 non-null  object 
 2   Region                              234623 non-null  object 
 3   City,Town,Ward,Village code         349894 non-null  int64  
 4   Prefecture                          349894 non-null  object 
 5   City,Town,Ward,Village              349894 non-null  object 
 6   Area                                348523 non-null  object 
 7   Nearest stationFName               347236 non-null  object 
 8   Nearest stationFDistance(minute)   343175 non-null  object 
 9   Transaction-price(total)            349894 non-null  int64  
 10  Layout                              109460 non-null  object 
 11  Area(m^2)                 

In [105]:
kg['Transaction-price(total)'].describe()

count    3.498940e+05
mean     4.087586e+07
std      1.715685e+08
min      1.000000e+02
25%      1.800000e+07
50%      2.900000e+07
75%      4.200000e+07
max      3.800000e+10
Name: Transaction-price(total), dtype: float64

In [106]:
kg.isna().sum().sort_values(ascending=False)[:28] / kg.shape[0]

Transactional factors                 0.949928
Transaction-price(Unit price m^2)     0.751608
Renovation                            0.705908
Layout                                0.687162
Total floor area(m^2)                 0.595578
Purpose of Use                        0.529778
Frontage                              0.390478
Frontage roadFBreadth(m)             0.340146
Frontage roadFClassification         0.337071
Land shape                            0.330426
Frontage roadFDirection              0.330231
Region                                0.329445
Use                                   0.317205
Building structure                    0.295515
Year of construction                  0.286955
Maximus Floor-area Ratio(%)           0.035668
Maximus Building Coverage Ratio(%)    0.035668
City Planning                         0.023907
Nearest stationFDistance(minute)     0.019203
Nearest stationFName                 0.007597
Area                                  0.003918
City,Town,War

In [107]:
# removal of columns with excessive Nulls
kgn = kg.drop(['No','Layout','Transaction-price(Unit price m^2)','Total floor area(m^2)','Renovation','Transactional factors','Frontage','Purpose of Use','Frontage roadFBreadth(m)','Frontage roadFClassification','Frontage roadFDirection','Land shape','Region'],axis=1)


In [108]:
kgn.dropna(axis=0,inplace=True)

In [109]:
kgn.isna().sum().sort_values(ascending=False)

Type                                  0
City,Town,Ward,Village code           0
Prefecture                            0
City,Town,Ward,Village                0
Area                                  0
Nearest stationFName                 0
Nearest stationFDistance(minute)     0
Transaction-price(total)              0
Area(m^2)                             0
Year of construction                  0
Building structure                    0
Use                                   0
City Planning                         0
Maximus Building Coverage Ratio(%)    0
Maximus Floor-area Ratio(%)           0
Transaction period                    0
dtype: int64

In [110]:
kgn.shape

(222478, 16)

In [111]:
kgn.dtypes

Type                                   object
City,Town,Ward,Village code             int64
Prefecture                             object
City,Town,Ward,Village                 object
Area                                   object
Nearest stationFName                  object
Nearest stationFDistance(minute)      object
Transaction-price(total)                int64
Area(m^2)                              object
Year of construction                   object
Building structure                     object
Use                                    object
City Planning                          object
Maximus Building Coverage Ratio(%)    float64
Maximus Floor-area Ratio(%)           float64
Transaction period                     object
dtype: object

In [112]:
# Column Renaming
kgn.rename(columns = {'Transaction period':'Period'}, inplace = True)

In [113]:
kgn.Period.value_counts()

3rd quarter 2015    4103
1st quarter 2015    4086
1st quarter 2016    4024
1st quarter 2014    3910
2nd quarter 2015    3888
                    ... 
3rd quarter 2006    1390
2nd quarter 2006    1223
4th quarter 2005    1121
1st quarter 2006    1120
3rd quarter 2005    1081
Name: Period, Length: 70, dtype: int64

In [114]:
#Column splitting
kgn['Period'].str.split('quarter',expand=True).rename({0:'quarter', 1:'year'}, axis = 1)

,quarter,year
1,4th,2010
2,1st,2010
6,4th,2010
7,4th,2010
8,3rd,2010
...,...,...
349878,4th,2008
349879,2nd,2008
349882,4th,2007
349884,4th,2007


In [115]:
#splitting period column to format it properly
kgn[['quarter','year']]=kgn['Period'].str.split('quarter',expand=True)

In [116]:
# reg[x] to remove a digit from a string in Quarter
kgn['quarter']=kgn['quarter'].str.extract('(\d)', expand=False)

In [117]:
#removal of spaces
kgn['year']=kgn['year'].str.strip()

In [118]:
#combined column
kgn['Period'] = kgn['year'] + '_q' + kgn['quarter']

In [119]:
#value check- 2000 or greater is a string therefore is not sorted right
kgn['Area(m^2)'].sort_values()

21296      10
4667       10
12634      10
54034      10
15857      10
         ... 
302254    990
202639    990
117148    990
35470     990
56495     990
Name: Area(m^2), Length: 222478, dtype: object

In [120]:
#replacing string values based on assumption 
kgn['Area(m^2)'].replace('2,000 m^2 or greater.', '2000',inplace=True)


In [121]:
#casting columns to perform calculations
kgn = kgn.astype({'Transaction-price(total)': float,'Area(m^2)': float})

In [122]:
#checkup
kgn['Area(m^2)'].sort_values()

36792       10.0
167832      10.0
15947       10.0
30177       10.0
31068       10.0
           ...  
143291    2000.0
240857    2000.0
302270    2000.0
304598    2000.0
147571    2000.0
Name: Area(m^2), Length: 222478, dtype: float64

In [123]:
#calculated field for price per meter squared
kgn['Price_Per_M^2'] = kgn['Transaction-price(total)']/kgn['Area(m^2)']

In [124]:
#checkup
kgn

,Type,"City,Town,Ward,Village code",Prefecture,"City,Town,Ward,Village",Area,Nearest stationFName,Nearest stationFDistance(minute),Transaction-price(total),Area(m^2),Year of construction,Building structure,Use,City Planning,Maximus Building Coverage Ratio(%),Maximus Floor-area Ratio(%),Period,quarter,year,Price_Per_M^2
1,Residential Land(Land and Building),14101,Kanagawa Prefecture,"Tsurumi Ward,Yokohama City",Asahicho,Anzen,7,38000000.0,65.0,2010,W,House,Quasi-industrial Zone,60.0,200.0,2010_q4,4,2010,584615.384615
2,"Pre-owned Condominiums, etc.",14101,Kanagawa Prefecture,"Tsurumi Ward,Yokohama City",Asahicho,Anzen,11,22000000.0,80.0,1991,RC,House,Quasi-industrial Zone,60.0,200.0,2010_q1,1,2010,275000.000000
6,"Pre-owned Condominiums, etc.",14101,Kanagawa Prefecture,"Tsurumi Ward,Yokohama City",Asahicho,Tsurumi,24,24000000.0,70.0,1999,RC,House,Quasi-industrial Zone,60.0,200.0,2010_q4,4,2010,342857.142857
7,"Pre-owned Condominiums, etc.",14101,Kanagawa Prefecture,"Tsurumi Ward,Yokohama City",Asahicho,Tsurumi,24,25000000.0,60.0,1999,RC,House,Quasi-industrial Zone,60.0,200.0,2010_q4,4,2010,416666.666667
8,Residential Land(Land and Building),14101,Kanagawa Prefecture,"Tsurumi Ward,Yokohama City",Asahicho,Tsurumi,30-60minutes,37000000.0,60.0,2010,W,House,Quasi-industrial Zone,60.0,200.0,2010_q3,3,2010,616666.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
349878,Residential Land(Land and Building),14402,Kanagawa Prefecture,"Kiyokawa Village,Aiko County",Susugaya,Hon-atsugi,2H-,5300000.0,430.0,1988,S,"House, Office",Outside City Planning Area,50.0,100.0,2008_q4,4,2008,12325.581395
349879,Residential Land(Land and Building),14402,Kanagawa Prefecture,"Kiyokawa Village,Aiko County",Susugaya,Hon-atsugi,2H-,25000000.0,165.0,before the war,W,House,Outside City Planning Area,50.0,100.0,2008_q2,2,2008,151515.151515
349882,Residential Land(Land and Building),14402,Kanagawa Prefecture,"Kiyokawa Village,Aiko County",Susugaya,Hon-atsugi,2H-,34000000.0,250.0,2007,W,House,Outside City Planning Area,50.0,100.0,2007_q4,4,2007,136000.000000
349884,Residential Land(Land and Building),14402,Kanagawa Prefecture,"Kiyokawa Village,Aiko County",Susugaya,Hon-atsugi,2H-,8000000.0,650.0,1990,S,House,Outside City Planning Area,50.0,100.0,2007_q4,4,2007,12307.692308


In [125]:
#checkup for calculated field
kgn['Price_Per_M^2']


1         584615.384615
2         275000.000000
6         342857.142857
7         416666.666667
8         616666.666667
              ...      
349878     12325.581395
349879    151515.151515
349882    136000.000000
349884     12307.692308
349885    150000.000000
Name: Price_Per_M^2, Length: 222478, dtype: float64

In [126]:
#overview of prices to determine classifications of properties
kgn['Transaction-price(total)'].describe()

count    2.224780e+05
mean     3.907726e+07
std      1.354334e+08
min      5.000000e+02
25%      1.900000e+07
50%      3.100000e+07
75%      4.200000e+07
max      3.800000e+10
Name: Transaction-price(total), dtype: float64

In [127]:
#Classification of properties function


def property_type_func(row, q1,q3):


    if row['Transaction-price(total)'] <= q1:
           return 'low_end'
    elif row['Transaction-price(total)'] >= q3:
            return 'high_end'
    else:
            return 'mid_end'

In [128]:
#Applying function to data
q1 = kgn['Transaction-price(total)'].quantile(0.25)
q3 = kgn['Transaction-price(total)'].quantile(0.75)

kgn['Property_Classification']=kgn.apply(property_type_func,args=(q1,q3), axis=1)

In [129]:
#checkup
kgn['Property_Classification'].value_counts()

mid_end     105831
high_end     59552
low_end      57095
Name: Property_Classification, dtype: int64

In [130]:
#checkup
kgn

,Type,"City,Town,Ward,Village code",Prefecture,"City,Town,Ward,Village",Area,Nearest stationFName,Nearest stationFDistance(minute),Transaction-price(total),Area(m^2),Year of construction,Building structure,Use,City Planning,Maximus Building Coverage Ratio(%),Maximus Floor-area Ratio(%),Period,quarter,year,Price_Per_M^2,Property_Classification
1,Residential Land(Land and Building),14101,Kanagawa Prefecture,"Tsurumi Ward,Yokohama City",Asahicho,Anzen,7,38000000.0,65.0,2010,W,House,Quasi-industrial Zone,60.0,200.0,2010_q4,4,2010,584615.384615,mid_end
2,"Pre-owned Condominiums, etc.",14101,Kanagawa Prefecture,"Tsurumi Ward,Yokohama City",Asahicho,Anzen,11,22000000.0,80.0,1991,RC,House,Quasi-industrial Zone,60.0,200.0,2010_q1,1,2010,275000.000000,mid_end
6,"Pre-owned Condominiums, etc.",14101,Kanagawa Prefecture,"Tsurumi Ward,Yokohama City",Asahicho,Tsurumi,24,24000000.0,70.0,1999,RC,House,Quasi-industrial Zone,60.0,200.0,2010_q4,4,2010,342857.142857,mid_end
7,"Pre-owned Condominiums, etc.",14101,Kanagawa Prefecture,"Tsurumi Ward,Yokohama City",Asahicho,Tsurumi,24,25000000.0,60.0,1999,RC,House,Quasi-industrial Zone,60.0,200.0,2010_q4,4,2010,416666.666667,mid_end
8,Residential Land(Land and Building),14101,Kanagawa Prefecture,"Tsurumi Ward,Yokohama City",Asahicho,Tsurumi,30-60minutes,37000000.0,60.0,2010,W,House,Quasi-industrial Zone,60.0,200.0,2010_q3,3,2010,616666.666667,mid_end
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
349878,Residential Land(Land and Building),14402,Kanagawa Prefecture,"Kiyokawa Village,Aiko County",Susugaya,Hon-atsugi,2H-,5300000.0,430.0,1988,S,"House, Office",Outside City Planning Area,50.0,100.0,2008_q4,4,2008,12325.581395,low_end
349879,Residential Land(Land and Building),14402,Kanagawa Prefecture,"Kiyokawa Village,Aiko County",Susugaya,Hon-atsugi,2H-,25000000.0,165.0,before the war,W,House,Outside City Planning Area,50.0,100.0,2008_q2,2,2008,151515.151515,mid_end
349882,Residential Land(Land and Building),14402,Kanagawa Prefecture,"Kiyokawa Village,Aiko County",Susugaya,Hon-atsugi,2H-,34000000.0,250.0,2007,W,House,Outside City Planning Area,50.0,100.0,2007_q4,4,2007,136000.000000,mid_end
349884,Residential Land(Land and Building),14402,Kanagawa Prefecture,"Kiyokawa Village,Aiko County",Susugaya,Hon-atsugi,2H-,8000000.0,650.0,1990,S,House,Outside City Planning Area,50.0,100.0,2007_q4,4,2007,12307.692308,low_end


In [131]:
kgn.rename(columns = {'Nearest station\x81FDistance(minute)':'Distance from Nearest Station(minute)','Nearest stationFName':'Nearest Station Name','Area':'District','City,Town,Ward,Village code':'Ward/City Code','City,Town,Ward,Village':'Ward/City','Transaction-price(total)':'Transaction-price(¥)','Maximus Building Coverage Ratio(%)':'Coverage Ratio(%)','Maximus Floor-area Ratio(%)':'Floor-area Ratio(%)'},inplace=True)

In [132]:
kgn

,Type,Ward/City Code,Prefecture,Ward/City,District,Nearest Station Name,Distance from Nearest Station(minute),Transaction-price(¥),Area(m^2),Year of construction,Building structure,Use,City Planning,Coverage Ratio(%),Floor-area Ratio(%),Period,quarter,year,Price_Per_M^2,Property_Classification
1,Residential Land(Land and Building),14101,Kanagawa Prefecture,"Tsurumi Ward,Yokohama City",Asahicho,Anzen,7,38000000.0,65.0,2010,W,House,Quasi-industrial Zone,60.0,200.0,2010_q4,4,2010,584615.384615,mid_end
2,"Pre-owned Condominiums, etc.",14101,Kanagawa Prefecture,"Tsurumi Ward,Yokohama City",Asahicho,Anzen,11,22000000.0,80.0,1991,RC,House,Quasi-industrial Zone,60.0,200.0,2010_q1,1,2010,275000.000000,mid_end
6,"Pre-owned Condominiums, etc.",14101,Kanagawa Prefecture,"Tsurumi Ward,Yokohama City",Asahicho,Tsurumi,24,24000000.0,70.0,1999,RC,House,Quasi-industrial Zone,60.0,200.0,2010_q4,4,2010,342857.142857,mid_end
7,"Pre-owned Condominiums, etc.",14101,Kanagawa Prefecture,"Tsurumi Ward,Yokohama City",Asahicho,Tsurumi,24,25000000.0,60.0,1999,RC,House,Quasi-industrial Zone,60.0,200.0,2010_q4,4,2010,416666.666667,mid_end
8,Residential Land(Land and Building),14101,Kanagawa Prefecture,"Tsurumi Ward,Yokohama City",Asahicho,Tsurumi,30-60minutes,37000000.0,60.0,2010,W,House,Quasi-industrial Zone,60.0,200.0,2010_q3,3,2010,616666.666667,mid_end
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
349878,Residential Land(Land and Building),14402,Kanagawa Prefecture,"Kiyokawa Village,Aiko County",Susugaya,Hon-atsugi,2H-,5300000.0,430.0,1988,S,"House, Office",Outside City Planning Area,50.0,100.0,2008_q4,4,2008,12325.581395,low_end
349879,Residential Land(Land and Building),14402,Kanagawa Prefecture,"Kiyokawa Village,Aiko County",Susugaya,Hon-atsugi,2H-,25000000.0,165.0,before the war,W,House,Outside City Planning Area,50.0,100.0,2008_q2,2,2008,151515.151515,mid_end
349882,Residential Land(Land and Building),14402,Kanagawa Prefecture,"Kiyokawa Village,Aiko County",Susugaya,Hon-atsugi,2H-,34000000.0,250.0,2007,W,House,Outside City Planning Area,50.0,100.0,2007_q4,4,2007,136000.000000,mid_end
349884,Residential Land(Land and Building),14402,Kanagawa Prefecture,"Kiyokawa Village,Aiko County",Susugaya,Hon-atsugi,2H-,8000000.0,650.0,1990,S,House,Outside City Planning Area,50.0,100.0,2007_q4,4,2007,12307.692308,low_end


In [133]:
kgn['Distance from Nearest Station(minute)'].unique()

array(['7', '11', '24', '30-60minutes', '28', '29', '6', '10', '5', '4',
       '3', '8', '2', '1', '12', '16', '13', '21', '18', '15', '14', '20',
       '19', '9', '26', '23', '25', '27', '17', '22', '0', '2H-',
       '1H-1H30', '1H30-2H'], dtype=object)

In [134]:
#estimate of minutes using average
kgn.replace({'Distance from Nearest Station(minute)' : {'30-60minutes' : '45', '2H-': '120', '1H-1H30' : '75','1H30-2H':'105'}},inplace=True)

In [135]:
kgn['Distance from Nearest Station(minute)'].unique()

array(['7', '11', '24', '45', '28', '29', '6', '10', '5', '4', '3', '8',
       '2', '1', '12', '16', '13', '21', '18', '15', '14', '20', '19',
       '9', '26', '23', '25', '27', '17', '22', '0', '120', '75', '105'],
      dtype=object)

In [136]:
kgn['Year of construction'].unique()

array(['2010', '1991', '1999', '2009', '1988', '1987', '1986', '1959',
       '1995', '2000', '2008', '1990', '1977', '1973', '1970', '1984',
       '2006', '2001', '2002', '2004', '1983', '1989', '1985', '1969',
       '1960', '1997', '1975', '2005', '1976', '1993', '2003', '1979',
       '1994', '2011', '1965', '1971', '1947', '1964', '1998', '2007',
       '1996', '1961', '1968', '1966', '1980', '1967', '1972', '1982',
       '1981', '1992', '1978', '2014', '2012', '2013', '2017', '2020',
       '2021', '2018', '2022', '2019', '1963', '1956', '1955', '1962',
       '1974', '1950', '2023', '2015', '2016', '1952', 'before the war',
       '1953', '1946', '1957', '1954', '1958', '1948', '1949', '1951'],
      dtype=object)

In [137]:
kgn.drop(kgn[kgn['Year of construction'] == 'before the war'].index, inplace = True)


In [138]:
kgn['Year of construction'].unique()

array(['2010', '1991', '1999', '2009', '1988', '1987', '1986', '1959',
       '1995', '2000', '2008', '1990', '1977', '1973', '1970', '1984',
       '2006', '2001', '2002', '2004', '1983', '1989', '1985', '1969',
       '1960', '1997', '1975', '2005', '1976', '1993', '2003', '1979',
       '1994', '2011', '1965', '1971', '1947', '1964', '1998', '2007',
       '1996', '1961', '1968', '1966', '1980', '1967', '1972', '1982',
       '1981', '1992', '1978', '2014', '2012', '2013', '2017', '2020',
       '2021', '2018', '2022', '2019', '1963', '1956', '1955', '1962',
       '1974', '1950', '2023', '2015', '2016', '1952', '1953', '1946',
       '1957', '1954', '1958', '1948', '1949', '1951'], dtype=object)

In [139]:
kgn.to_csv('Kanagawa_Real_Estate_Finals.csv',index=False)